## Module

In [1]:
import optuna
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Dataset

In [2]:
train = pd.read_csv('../input/titanic/train.csv')
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv('../input/titanic/test.csv')
print(test.shape)
test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## Info

In [5]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

## Preprocess

In [6]:
features = [
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked'
]

target = 'Survived'

In [7]:
for col in ['Sex', 'Embarked']:
    le = LabelEncoder()
    le.fit(train[col].fillna('nan'))
    
    train[col] = le.transform(train[col].fillna('nan'))
    test[col] = le.transform(test[col].fillna('nan'))

In [8]:
X_train = train[features]
y_train = train[target]
print(X_train.shape)
print(y_train.shape)

(891, 7)
(891,)


## Params

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=123)

print(X_train.shape)
print(y_valid.shape)
print(X_train.shape)
print(y_valid.shape)

(623, 7)
(268,)
(623, 7)
(268,)


In [10]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': trial.suggest_uniform('eta', 0.05, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_uniform('min_child_weight', 0.1, 10),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 0.95),
        'subsample': trial.suggest_uniform('subsample', 0.6, 0.95)
    }
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    num_round = 500
    
    model = xgb.train(
        params,
        dtrain,
        num_round,
        evals=watchlist,
        early_stopping_rounds=20
    )
    
    y_pred_valid = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [11]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

[I 2023-10-26 04:09:05,857] A new study created in memory with name: no-name-04bfea76-18fe-4e94-b5cb-7930235c1db8


[0]	train-logloss:0.65963	eval-logloss:0.66114
[1]	train-logloss:0.63142	eval-logloss:0.63393
[2]	train-logloss:0.60928	eval-logloss:0.61561
[3]	train-logloss:0.59797	eval-logloss:0.60414
[4]	train-logloss:0.57702	eval-logloss:0.58500
[5]	train-logloss:0.55828	eval-logloss:0.56847
[6]	train-logloss:0.54353	eval-logloss:0.55596
[7]	train-logloss:0.52934	eval-logloss:0.54306
[8]	train-logloss:0.51661	eval-logloss:0.53081
[9]	train-logloss:0.50534	eval-logloss:0.52001
[10]	train-logloss:0.49517	eval-logloss:0.51184
[11]	train-logloss:0.48656	eval-logloss:0.50588
[12]	train-logloss:0.47780	eval-logloss:0.49858
[13]	train-logloss:0.47010	eval-logloss:0.49161
[14]	train-logloss:0.46451	eval-logloss:0.48725
[15]	train-logloss:0.45848	eval-logloss:0.48165
[16]	train-logloss:0.45272	eval-logloss:0.47655
[17]	train-logloss:0.44673	eval-logloss:0.47205
[18]	train-logloss:0.44312	eval-logloss:0.47004
[19]	train-logloss:0.43811	eval-logloss:0.46680
[20]	train-logloss:0.43431	eval-logloss:0.46376
[2

[I 2023-10-26 04:09:06,148] Trial 0 finished with value: 0.43048525290257894 and parameters: {'eta': 0.07744067519636624, 'max_depth': 8, 'min_child_weight': 8.458230910952071, 'colsample_bytree': 0.9002809661679648, 'subsample': 0.8965381085744438}. Best is trial 0 with value: 0.43048525290257894.


[0]	train-logloss:0.65666	eval-logloss:0.65974
[1]	train-logloss:0.62557	eval-logloss:0.63191
[2]	train-logloss:0.59873	eval-logloss:0.60933
[3]	train-logloss:0.58566	eval-logloss:0.59926
[4]	train-logloss:0.56254	eval-logloss:0.57815
[5]	train-logloss:0.54200	eval-logloss:0.56042
[6]	train-logloss:0.52495	eval-logloss:0.54799
[7]	train-logloss:0.50917	eval-logloss:0.53267
[8]	train-logloss:0.49476	eval-logloss:0.51941
[9]	train-logloss:0.48230	eval-logloss:0.50747
[10]	train-logloss:0.47110	eval-logloss:0.49878
[11]	train-logloss:0.46161	eval-logloss:0.49156
[12]	train-logloss:0.45288	eval-logloss:0.48356
[13]	train-logloss:0.44444	eval-logloss:0.47612
[14]	train-logloss:0.43799	eval-logloss:0.47192
[15]	train-logloss:0.43153	eval-logloss:0.46472
[16]	train-logloss:0.42519	eval-logloss:0.45994
[17]	train-logloss:0.41894	eval-logloss:0.45466
[18]	train-logloss:0.41312	eval-logloss:0.45171
[19]	train-logloss:0.40808	eval-logloss:0.44826
[20]	train-logloss:0.40353	eval-logloss:0.44635
[2

[I 2023-10-26 04:09:06,333] Trial 1 finished with value: 0.4196977114110296 and parameters: {'eta': 0.08117818483929862, 'max_depth': 5, 'min_child_weight': 4.432113391500656, 'colsample_bytree': 0.912120550273728, 'subsample': 0.9372819661753602}. Best is trial 1 with value: 0.4196977114110296.


...

[I 2023-10-26 04:09:17,466] Trial 37 finished with value: 0.42857610995867357 and parameters: {'eta': 0.05858148386307203, 'max_depth': 5, 'min_child_weight': 4.946534475221827, 'colsample_bytree': 0.7186447908729223, 'subsample': 0.6628215913212495}. Best is trial 9 with value: 0.4196600303568168.


[0]	train-logloss:0.66507	eval-logloss:0.66811
[1]	train-logloss:0.64141	eval-logloss:0.64645
[2]	train-logloss:0.62221	eval-logloss:0.62970
[3]	train-logloss:0.61084	eval-logloss:0.62036
[4]	train-logloss:0.60048	eval-logloss:0.61047
[5]	train-logloss:0.58242	eval-logloss:0.59510
[6]	train-logloss:0.56415	eval-logloss:0.58047
[7]	train-logloss:0.54827	eval-logloss:0.56586
[8]	train-logloss:0.53973	eval-logloss:0.55891
[9]	train-logloss:0.52614	eval-logloss:0.54662
[10]	train-logloss:0.51281	eval-logloss:0.53652
[11]	train-logloss:0.50557	eval-logloss:0.53194
[12]	train-logloss:0.49413	eval-logloss:0.52195
[13]	train-logloss:0.48822	eval-logloss:0.51682
[14]	train-logloss:0.48276	eval-logloss:0.51318
[15]	train-logloss:0.47322	eval-logloss:0.50616
[16]	train-logloss:0.46374	eval-logloss:0.49867
[17]	train-logloss:0.45441	eval-logloss:0.49155
[18]	train-logloss:0.44752	eval-logloss:0.48608
[19]	train-logloss:0.43986	eval-logloss:0.47915
[20]	train-logloss:0.43266	eval-logloss:0.47313
[2

[I 2023-10-26 04:09:17,739] Trial 38 finished with value: 0.419781346610789 and parameters: {'eta': 0.058549329934156576, 'max_depth': 5, 'min_child_weight': 2.3168544117977636, 'colsample_bytree': 0.7208730882439159, 'subsample': 0.9248284527129568}. Best is trial 9 with value: 0.4196600303568168.


[0]	train-logloss:0.65969	eval-logloss:0.66242
[1]	train-logloss:0.62948	eval-logloss:0.63350
[2]	train-logloss:0.60824	eval-logloss:0.61556
[3]	train-logloss:0.59639	eval-logloss:0.60405
[4]	train-logloss:0.58403	eval-logloss:0.59127
[5]	train-logloss:0.56630	eval-logloss:0.57679
[6]	train-logloss:0.55009	eval-logloss:0.56293
[7]	train-logloss:0.53408	eval-logloss:0.54841
[8]	train-logloss:0.52829	eval-logloss:0.54401
[9]	train-logloss:0.51540	eval-logloss:0.53250
[10]	train-logloss:0.50446	eval-logloss:0.52218
[11]	train-logloss:0.49831	eval-logloss:0.51765
[12]	train-logloss:0.48853	eval-logloss:0.50763
[13]	train-logloss:0.48440	eval-logloss:0.50564
[14]	train-logloss:0.48011	eval-logloss:0.50170
[15]	train-logloss:0.47363	eval-logloss:0.49627
[16]	train-logloss:0.46755	eval-logloss:0.49097
[17]	train-logloss:0.46113	eval-logloss:0.48524
[18]	train-logloss:0.45607	eval-logloss:0.48194
[19]	train-logloss:0.45088	eval-logloss:0.47700
[20]	train-logloss:0.44601	eval-logloss:0.47300
[2

[I 2023-10-26 04:09:18,146] Trial 39 finished with value: 0.42530741802514044 and parameters: {'eta': 0.08522072009617665, 'max_depth': 4, 'min_child_weight': 7.858078028488696, 'colsample_bytree': 0.7750092042052554, 'subsample': 0.6176295198651403}. Best is trial 9 with value: 0.4196600303568168.


In [12]:
params = {
    'objective': 'binary:logistic'
}

for i, j in study.best_params.items():
    params[i] = j

params

{'objective': 'binary:logistic',
 'eta': 0.07185159768996707,
 'max_depth': 4,
 'min_child_weight': 1.0828754685538413,
 'colsample_bytree': 0.939433173711362,
 'subsample': 0.8285990125292781}

## Model

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
num_round = 500
    
model = xgb.train(
    params,
    dtrain,
    num_round,
    evals=watchlist,
    early_stopping_rounds=20
)

[04:09:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.66105	eval-logloss:0.66319
[1]	train-logloss:0.63318	eval-logloss:0.63856
[2]	train-logloss:0.60791	eval-logloss:0.61605
[3]	train-logloss:0.59455	eval-logloss:0.60638
[4]	train-logloss:0.57233	eval-logloss:0.58544
[5]	train-logloss:0.55267	eval-logloss:0.56793
[6]	train-logloss:0.53465	eval-logloss:0.55422
[7]	train-logloss:0.51982	eval-logloss:0.54096
[8]	train-logloss:0.50548	eval-logloss:0.52668
[9]	train-logloss:0.49207	eval-logloss:0.51407
[10]	train-logloss:0.47909	eval-logloss:0.50437
[11]	train-logloss:0.46797	eval-logloss:0.49442
[12]	train-logloss:0.45695	eval-logloss:0.48439
[13]	train-logloss:0.44731	eval-logloss:0.47701
[14]	train-logloss:0.44210	eval-logloss:0.47611
[15]	train-logloss:0.43516	eval-loglos

## Evaluate

In [14]:
y_pred_valid = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
y_pred_valid = (y_pred_valid > 0.5).astype(int)

In [15]:
f1_score(y_valid, y_pred_valid, average='macro')

0.7951821961359746

## Submit

In [16]:
X_test = test[features]

In [17]:
dtest = xgb.DMatrix(X_test)
y_pred_test = model.predict(dtest, ntree_limit=model.best_ntree_limit)
y_pred_test = (y_pred_test > 0.5).astype(int)

In [18]:
submit = pd.DataFrame(y_pred_test, index=test['PassengerId'], columns=['Survived'])
submit.to_csv('submit.csv')